In [1]:
# 导入所需库
import serial
import serial.tools.list_ports
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, Button, Dropdown, FloatSlider, IntSlider, Checkbox, Text
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import numpy as np
import time
from threading import Thread
import json


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app.start()
  File "/usr/local/share/pynq-venv/lib

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/share/pynq-venv/lib/python3.10/site-packages/traitlets/config/application.py", line 976, in launch_instance
    app.start()
  File "/usr/local/share/pynq-venv/lib

AttributeError: _ARRAY_API not found

In [2]:
# 全局变量初始化
global serial_cfg  # 配置端口（CFG_port）
global serial_data # 数据端口（Data_port）
global radar_config# 雷达配置参数字典
global is_connected# 串口连接状态
global is_recording# 数据录制状态
global plot_data   # 待绘制数据缓存

# 初始化状态变量
serial_cfg = None
serial_data = None
is_connected = False
is_recording = False
plot_data = {
    "xy_scatter": [],    # X-Y 散点图数据
    "doppler_range": [], # 多普勒-距离图数据
    "range_profile": [], # 距离剖面图数据
    "cpu_load": []       # CPU 负载数据
}

In [3]:
# 雷达默认配置参数（基于 IWR6843AOP 特性）
radar_config = {
    "platform": "IWR6843AOP",
    "sdk_version": "3.6.0.0",
    "antenna_config": "4R_3T_30deg",  # 天线配置（4接收，3发射，30°方位,30°高成分辨率）
    "freq_band": "60-64GHz",          # 频率带宽（60-64GHz 宽带）
    "desirable_config": "Best_Range_Resolution", # 优先配置目标
    "frame_rate": 10,                 # 帧速率（fps）
    "range_resolution": 0.04,         # 距离分辨率（m）
    "max_unambiguous_range": 9.0,    # 最大无模糊距离（m）
    "max_radial_velocity": 1.0,       # 最大径向速度（m/s）
    "radial_velocity_res": 0.13,     # 径向速度分辨率（m/s）
    "selected_plots": {               # 选中的绘图类型
        "scatter_plot": True,
        "range_profile": True,
        "noise_profile": True,
        "range_doppler_heatmap": False,
        "range_azimuth_heatmap": False,
        "cpu_stats": True
    }
}

In [4]:
# 支持的配置选项（与 IWR6843AOP 硬件匹配）
ANTENNA_OPTIONS = [
    ("4R,3T (30° Azimuth/lev)", "4R_3T_30deg"),
    ("4R,2T (15° Azimuth)", "4R_2T_15deg"),
    ("4R,2T (7.5° Azimuth)", "4R_2T_7.5deg")
]

FREQ_BAND_OPTIONS = [
    ("60-64 GHz (Wide Band)", "60-64GHz")
]

DESIRABLE_CONFIG_OPTIONS = [
    ("Best Range Resolution", "Best_Range_Resolution"),
    ("Best Velocity Resolution", "Best_Velocity_Resolution"),
    ("Best Range", "Best_Range")
]

SDK_VERSION_OPTIONS = [
    ("3.6.0.0", "3.6.0.0"),
    ("3.5.0.2", "3.5.0.2"),
    ("3.4.0.2", "3.4.0.2"),
    ("3.3.0.1", "3.3.0.1"),
]

In [5]:
def scan_serial_ports():
    """扫描当前可用的串口端口，返回端口列表"""
    ports = serial.tools.list_ports.comports()
    port_list = [port.device for port in ports]
    return port_list if port_list else ["No Ports Found"]

def connect_serial(cfg_port, data_port, cfg_baud=115200, data_baud=921600):
    """
    连接雷达的 CFG_port 和 Data_port
    :param cfg_port: 配置端口（如 COM3）
    :param data_port: 数据端口（如 COM4）
    :param cfg_baud: 配置端口波特率（默认 115200）
    :param data_baud: 数据端口波特率（默认 921600）
    :return: 连接状态（True/False）
    """
    global serial_cfg, serial_data, is_connected
    try:
        # 关闭已存在的连接
        if serial_cfg and serial_cfg.is_open:
            serial_cfg.close()
        if serial_data and serial_data.is_open:
            serial_data.close()
        
        # 连接 CFG_port（CLI 命令端口）
        serial_cfg = serial.Serial(
            port=cfg_port,
            baudrate=cfg_baud,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=1
        )
        
        # 连接 Data_port（数据接收端口）
        serial_data = serial.Serial(
            port=data_port,
            baudrate=data_baud,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=0.1
        )
        
        time.sleep(2)  # 等待连接稳定
        is_connected = True
        print(f"Successfully connected to CFG_port: {cfg_port}, Data_port: {data_port}")
        return True
    except Exception as e:
        print(f"Serial connection failed: {str(e)}")
        is_connected = False
        return False

def disconnect_serial():
    """断开串口连接"""
    global serial_cfg, serial_data, is_connected
    try:
        if serial_cfg and serial_cfg.is_open:
            serial_cfg.close()
        if serial_data and serial_data.is_open:
            serial_data.close()
        is_connected = False
        print("Serial ports disconnected")
        return True
    except Exception as e:
        print(f"Disconnect failed: {str(e)}")
        return False

def send_cli_command(cmd):
    """
    向 CFG_port 发送 CLI 命令
    :param cmd: 命令字符串（如 "sensorStart"）
    :return: 设备响应结果
    """
    global serial_cfg, is_connected
    if not is_connected or not serial_cfg.is_open:
        return "Error: Serial port not connected"
    
    try:
        # CLI 命令需以换行符结尾
        cmd = cmd.strip() + "\r\n"
        serial_cfg.write(cmd.encode("utf-8"))
        time.sleep(0.5)  # 等待响应
        response = serial_cfg.read_all().decode("utf-8").strip()
        return f"Command: {cmd.strip()}\nResponse: {response}"
    except Exception as e:
        return f"Command send failed: {str(e)}"

def parse_data_port():
    """
    解析 Data_port 接收的二进制数据（模拟 IWR6843AOP 输出格式）
    实际项目需根据雷达 SDK 数据协议调整解析逻辑
    """
    global serial_data, plot_data, is_recording
    while is_connected and serial_data.is_open:
        try:
            if serial_data.in_waiting > 0:
                # 读取数据（模拟：实际需按雷达数据帧格式解析）
                data = serial_data.read(128)  # 读取 128 字节数据
                if data:
                    # 模拟生成雷达数据（实际需替换为真实解析逻辑）
                    timestamp = time.time()
                    num_points = np.random.randint(5, 20)  # 随机生成 5-20 个目标点
                    
                    # 1. X-Y 散点图数据（x: 横向距离，y: 纵向距离）
                    xy_data = {
                        "timestamp": timestamp,
                        "x": np.random.uniform(-5, 5, num_points).tolist(),  # -5~5m
                        "y": np.random.uniform(0, 10, num_points).tolist()   # 0~10m
                    }
                    plot_data["xy_scatter"].append(xy_data)
                    
                    # 2. 多普勒-距离数据（range: 距离，doppler: 多普勒速度）
                    doppler_range_data = {
                        "timestamp": timestamp,
                        "range": np.random.uniform(0, 10, num_points).tolist(),  # 0~10m
                        "doppler": np.random.uniform(-3, 3, num_points).tolist() # -3~3m/s
                    }
                    plot_data["doppler_range"].append(doppler_range_data)
                    
                    # 3. 距离剖面图数据（range_bin: 距离单元，power: 信号功率）
                    range_bins = np.arange(0, 10, 0.04).tolist()  # 0~10m，步长 0.04m
                    power = np.random.normal(-60, 10, len(range_bins)).tolist()  # 功率（dB）
                    range_profile_data = {
                        "timestamp": timestamp,
                        "range_bin": range_bins,
                        "power": power
                    }
                    plot_data["range_profile"].append(range_profile_data)
                    
                    # 4. CPU 负载数据（active: 活跃期负载，interframe: 帧间隙负载）
                    cpu_load_data = {
                        "timestamp": timestamp,
                        "active": np.random.uniform(20, 60),  # 20~60%
                        "interframe": np.random.uniform(5, 15) # 5~15%
                    }
                    plot_data["cpu_load"].append(cpu_load_data)
                    
                    # 限制数据缓存大小（仅保留最近 100 帧）
                    for key in plot_data:
                        if len(plot_data[key]) > 100:
                            plot_data[key] = plot_data[key][-100:]
                    
                    # 若开启录制，将数据写入文件
                    if is_recording:
                        with open("radar_data.log", "a") as f:
                            f.write(json.dumps({
                                "timestamp": timestamp,
                                "xy_scatter": xy_data,
                                "doppler_range": doppler_range_data,
                                "range_profile": range_profile_data,
                                "cpu_load": cpu_load_data
                            }) + "\n")
        except Exception as e:
            print(f"Data parse error: {str(e)}")
            time.sleep(0.1)
        time.sleep(0.05)  # 控制数据解析频率

In [6]:
def create_config_ui():
    """创建雷达配置界面"""
    # 1. 串口端口选择
    port_list = scan_serial_ports()
    cfg_port_dropdown = Dropdown(
        options=port_list,
        description="CFG Port:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="200px")
    )
    data_port_dropdown = Dropdown(
        options=port_list,
        description="Data Port:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="200px")
    )
    
    # 刷新端口按钮
    def refresh_ports(_):
        new_ports = scan_serial_ports()
        cfg_port_dropdown.options = new_ports
        data_port_dropdown.options = new_ports
    refresh_port_btn = Button(description="Refresh Ports")
    refresh_port_btn.on_click(refresh_ports)
    
    # 波特率选择
    cfg_baud_dropdown = Dropdown(
        options=[115200, 230400],
        value=115200,
        description="CFG Baud:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="150px")
    )
    data_baud_dropdown = Dropdown(
        options=[921600, 1834000, 3125000, ("Custom", "custom")],
        value=921600,
        description="Data Baud:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="180px")
    )
    custom_baud_text = Text(
        value="",
        placeholder="Enter custom baud",
        description="Custom:",
        disabled=True,
        layout=widgets.Layout(width="150px")
    )
    
    # 自定义波特率启用逻辑
    def toggle_custom_baud(_):
        if data_baud_dropdown.value == "custom":
            custom_baud_text.disabled = False
        else:
            custom_baud_text.disabled = True
            custom_baud_text.value = ""
    data_baud_dropdown.observe(toggle_custom_baud, "value")
    
    # 连接/断开按钮
    connect_btn = Button(description="Connect", button_style="success")
    disconnect_btn = Button(description="Disconnect", button_style="danger")
    connection_status = Label(value="Status: Disconnected", color="red")
    
    # 连接按钮回调
    def on_connect(_):
        data_baud = data_baud_dropdown.value
        if data_baud == "custom":
            try:
                data_baud = int(custom_baud_text.value)
                if data_baud > 3125000:
                    connection_status.value = "Error: Max baud is 3125000"
                    return
            except ValueError:
                connection_status.value = "Error: Invalid custom baud"
                return
        
        success = connect_serial(
            cfg_port=cfg_port_dropdown.value,
            data_port=data_port_dropdown.value,
            cfg_baud=cfg_baud_dropdown.value,
            data_baud=data_baud
        )
        if success:
            connection_status.value = f"Status: Connected (CFG:{cfg_port_dropdown.value}, Data:{data_port_dropdown.value})"
            connection_status.style.color = "green"
            # 启动数据解析线程
            Thread(target=parse_data_port, daemon=True).start()
        else:
            connection_status.value = "Status: Connection Failed"
            connection_status.style.color = "red"
    
    # 断开按钮回调
    def on_disconnect(_):
        disconnect_serial()
        connection_status.value = "Status: Disconnected"
        connection_status.style.color = "red"
    
    connect_btn.on_click(on_connect)
    disconnect_btn.on_click(on_disconnect)
    
    # 2. 雷达核心参数配置
    # 平台与 SDK 版本
    platform_dropdown = Dropdown(
        options=["IWR6843AOP"],
        value="IWR6843AOP",
        description="Platform:",
        style={"description_width": "initial"},
        disabled=True,
        layout=widgets.Layout(width="200px")
    )
    sdk_version_dropdown = Dropdown(
        options=SDK_VERSION_OPTIONS,
        value="3.6.0.0",
        description="SDK Version:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="200px")
    )
    
    # 天线配置
    antenna_dropdown = Dropdown(
        options=ANTENNA_OPTIONS,
        value="4R_3T_30deg",
        description="Antenna Config:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    
    # 频率带宽
    freq_band_dropdown = Dropdown(
        options=FREQ_BAND_OPTIONS,
        value="60-64GHz",
        description="Frequency Band:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    
    # 优先配置目标
    desirable_config_dropdown = Dropdown(
        options=DESIRABLE_CONFIG_OPTIONS,
        value="Best_Range_Resolution",
        description="Desirable Config:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    
    # 帧速率（fps）
    frame_rate_slider = IntSlider(
        min=1, max=30, value=10, step=1,
        description="Frame Rate (fps):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )
    
    # 距离分辨率（m）
    range_res_slider = FloatSlider(
        min=0.02, max=0.2, value=0.04, step=0.001,
        description="Range Resolution (m):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )
    
    # 最大无模糊距离（m）
    max_range_slider = FloatSlider(
        min=1, max=30, value=10, step=0.1,
        description="Max Unambiguous Range (m):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )
    
    # 最大径向速度（m/s）
    max_vel_slider = FloatSlider(
        min=0.5, max=10, value=5, step=0.1,
        description="Max Radial Velocity (m/s):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )
    
    # 径向速度分辨率（m/s）
    vel_res_dropdown = Dropdown(
        options=[0.0625, 0.125, 0.25, 0.5],
        value=0.125,
        description="Radial Velocity Res (m/s):",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    
    # 3. 绘图选择
    plot_checkboxes = VBox([
        Checkbox(value=True, description="X-Y Scatter Plot", style={"description_width": "initial"}),
        Checkbox(value=True, description="Doppler-Range Plot", style={"description_width": "initial"}),
        Checkbox(value=True, description="Range Profile", style={"description_width": "initial"}),
        Checkbox(value=False, description="Range-Doppler Heatmap", style={"description_width": "initial"}),
        Checkbox(value=False, description="Range-Azimuth Heatmap", style={"description_width": "initial"}),
        Checkbox(value=True, description="CPU Load Stats", style={"description_width": "initial"})
    ])
    plot_label = Label(value="Select Plots to Display:", style={"font_weight": "bold"})
    plot_selection = VBox([plot_label, plot_checkboxes])
    
    # 4. 配置下发与保存
    send_config_btn = Button(description="Send Config to Radar", button_style="primary")
    save_config_btn = Button(description="Save Config to File", button_style="info")
    reset_config_btn = Button(description="Reset Config", button_style="warning")
    config_log = widgets.Textarea(
        value="",
        placeholder="Config log will appear here...",
        description="Config Log:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="800px", height="150px")
    )
    
    # 更新全局配置字典
    def update_radar_config(_):
        radar_config.update({
            "platform": platform_dropdown.value,
            "sdk_version": sdk_version_dropdown.value,
            "antenna_config": antenna_dropdown.value,
            "freq_band": freq_band_dropdown.value,
            "desirable_config": desirable_config_dropdown.value,
            "frame_rate": frame_rate_slider.value,
            "range_resolution": range_res_slider.value,
            "max_unambiguous_range": max_range_slider.value,
            "max_radial_velocity": max_vel_slider.value,
            "radial_velocity_res": vel_res_dropdown.value,
            "selected_plots": {
                "scatter_plot": plot_checkboxes.children[0].value,
                "doppler_range_plot": plot_checkboxes.children[1].value,
                "range_profile": plot_checkboxes.children[2].value,
                "range_doppler_heatmap": plot_checkboxes.children[3].value,
                "range_azimuth_heatmap": plot_checkboxes.children[4].value,
                "cpu_stats": plot_checkboxes.children[5].value
            }
        })
    
    # 下发配置按钮回调
    def on_send_config(_):
        if not is_connected:
            config_log.value += "Error: Serial port not connected. Cannot send config.\n"
            return
        
        # 更新配置并生成 CLI 命令（基于 IWR6843AOP 协议）
        update_radar_config(None)
        cli_commands = [
            "sensorStop",          # 停止雷达
            "flushCfg",            # 清空配置
            f"platform {radar_config['platform']}",  # 平台选择
            f"sdkVersion {radar_config['sdk_version']}",  # SDK 版本
            # 天线配置（示例命令，需根据实际协议调整）
            f"antennaCfg {radar_config['antenna_config']}",
            # 频率带宽配置
            f"freqBand {radar_config['freq_band']}",
            # 帧速率配置
            f"frameRate {radar_config['frame_rate']}",
            # 距离分辨率配置
            f"rangeRes {radar_config['range_resolution']}",
            # 最大无模糊距离配置
            f"maxRange {radar_config['max_unambiguous_range']}",
            # 最大径向速度配置
            f"maxVel {radar_config['max_radial_velocity']}",
            # 径向速度分辨率配置
            f"velRes {radar_config['radial_velocity_res']}",
            "sensorStart"          # 启动雷达
        ]
        
        # 发送命令并记录日志
        config_log.value = ""
        for cmd in cli_commands:
            response = send_cli_command(cmd)
            config_log.value += f"{response}\n\n"
            time.sleep(0.3)  # 等待命令执行
    
    # 保存配置按钮回调
    def on_save_config(_):
        update_radar_config(None)
        config_str = json.dumps(radar_config, indent=2)
        with open("radar_config.json", "w") as f:
            f.write(config_str)
        config_log.value += f"Config saved to radar_config.json\n{config_str}\n"
    
    # 重置配置按钮回调
    def on_reset_config(_):
        # 重置控件值为默认值
        sdk_version_dropdown.value = "3.6.0.0"
        antenna_dropdown.value = "4R_3T_30deg"
        freq_band_dropdown.value = "60-64GHz"
        desirable_config_dropdown.value = "Best_Range_Resolution"
        frame_rate_slider.value = 10
        range_res_slider.value = 0.04
        max_range_slider.value = 10
        max_vel_slider.value = 1
        vel_res_dropdown.value = 0.125
        for i, box in enumerate(plot_checkboxes.children):
            box.value = [True, True, True, False, False, True][i]
        config_log.value += "Config reset to default values\n"
    
    send_config_btn.on_click(on_send_config)
    save_config_btn.on_click(on_save_config)
    reset_config_btn.on_click(on_reset_config)
    
    # 组装配置界面
    serial_box = VBox([
        HBox([cfg_port_dropdown, data_port_dropdown, refresh_port_btn]),
        HBox([cfg_baud_dropdown, data_baud_dropdown, custom_baud_text]),
        HBox([connect_btn, disconnect_btn, connection_status])
    ])
    
    core_config_box = VBox([
        HBox([platform_dropdown, sdk_version_dropdown]),
        HBox([antenna_dropdown, freq_band_dropdown]),
        HBox([desirable_config_dropdown, vel_res_dropdown]),
        frame_rate_slider,
        range_res_slider,
        max_range_slider,
        max_vel_slider
    ])
    
    config_control_box = HBox([send_config_btn, save_config_btn, reset_config_btn])
    
    full_config_ui = VBox([
        Label(value="=== IWR6843AOP Radar Configuration ===", style={"font_size": "16px", "font_weight": "bold"}),
        serial_box,
        Label(value="--- Core Parameters ---", style={"font_weight": "bold"}),
        core_config_box,
        Label(value="--- Plot Selection ---", style={"font_weight": "bold"}),
        plot_selection,
        config_control_box,
        config_log
    ])
    
    return full_config_ui

# 显示配置界面
config_ui = create_config_ui()
display(config_ui)

In [7]:
def create_visualization_ui():
    """创建雷达数据可视化界面"""
    # 1. 可视化控制按钮
    start_plot_btn = Button(description="Start Plotting", button_style="success")
    stop_plot_btn = Button(description="Stop Plotting", button_style="danger")
    start_record_btn = Button(description="Start Recording", button_style="info")
    stop_record_btn = Button(description="Stop Recording", button_style="warning")
    plot_status = Label(value="Plot Status: Stopped", color="red")
    record_status = Label(value="Recording: Stopped", color="red")
    
    # 绘图状态变量
    global is_plotting
    is_plotting = False
    
    # 2. 创建子图布局（根据选中的绘图类型动态调整）
    fig = sp.make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "X-Y Scatter Plot (Detected Objects)",
            "Doppler-Range Plot",
            "Range Profile (0 Doppler)",
            "CPU Load (%)"
        ),
        shared_xaxes=False,
        shared_yaxes=False,
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    # 初始化子图（空数据）
    # 子图 1: X-Y 散点图
    fig.add_trace(go.Scatter(x=[], y=[], mode="markers", marker=dict(size=8, color="blue"), name="Objects"), row=1, col=1)
    fig.update_xaxes(title_text="Lateral Distance (m)", row=1, col=1)
    fig.update_yaxes(title_text="Longitudinal Distance (m)", row=1, col=1)
    
    # 子图 2: 多普勒-距离图
    fig.add_trace(go.Scatter(x=[], y=[], mode="markers", marker=dict(size=8, color="red"), name="Objects"), row=1, col=2)
    fig.update_xaxes(title_text="Range (m)", row=1, col=2)
    fig.update_yaxes(title_text="Doppler Velocity (m/s)", row=1, col=2)
    
    # 子图 3: 距离剖面图
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines", line=dict(color="blue"), name="Range Profile"), row=2, col=1)
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines", line=dict(color="green", dash="dash"), name="Noise Profile"), row=2, col=1)
    fig.update_xaxes(title_text="Range (m)", row=2, col=1)
    fig.update_yaxes(title_text="Power (dB)", row=2, col=1)
    
    # 子图 4: CPU 负载图
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines", line=dict(color="orange"), name="Active Load"), row=2, col=2)
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines", line=dict(color="purple", dash="dash"), name="Interframe Load"), row=2, col=2)
    fig.update_xaxes(title_text="Frame Index", row=2, col=2)
    fig.update_yaxes(title_text="CPU Load (%)", row=2, col=2, range=[0, 100])
    
    # 调整图大小
    fig.update_layout(height=800, width=1200, title_text="IWR6843AOP Radar Data Visualization")
    
    # 显示初始图
    plot_widget = go.FigureWidget(fig)
    
    # 3. 实时绘图函数
    def update_plots():
        """动态更新绘图数据"""
        global plot_data, is_plotting
        frame_idx = 0
        while is_plotting and is_connected:
            try:
                # 子图 1: X-Y 散点图（取最新一帧数据）
                if plot_data["xy_scatter"]:
                    latest_xy = plot_data["xy_scatter"][-1]
                    with plot_widget.batch_update():
                        plot_widget.data[0].x = latest_xy["x"]
                        plot_widget.data[0].y = latest_xy["y"]
                
                # 子图 2: 多普勒-距离图（取最新一帧数据）
                if plot_data["doppler_range"]:
                    latest_doppler = plot_data["doppler_range"][-1]
                    with plot_widget.batch_update():
                        plot_widget.data[1].x = latest_doppler["range"]
                        plot_widget.data[1].y = latest_doppler["doppler"]
                
                # 子图 3: 距离剖面图（取最新一帧数据）
                if plot_data["range_profile"]:
                    latest_range = plot_data["range_profile"][-1]
                    with plot_widget.batch_update():
                        plot_widget.data[2].x = latest_range["range_bin"]
                        plot_widget.data[2].y = latest_range["power"]
                        # 模拟噪声数据（比信号低 10dB）
                        noise_power = [p - 10 for p in latest_range["power"]]
                        plot_widget.data[3].x = latest_range["range_bin"]
                        plot_widget.data[3].y = noise_power
                
                # 子图 4: CPU 负载图（保留历史数据）
                if plot_data["cpu_load"]:
                    cpu_data = plot_data["cpu_load"][-50:]  # 保留最近 50 帧
                    timestamps = [d["timestamp"] for d in cpu_data]
                    active_loads = [d["active"] for d in cpu_data]
                    interframe_loads = [d["interframe"] for d in cpu_data]
                    with plot_widget.batch_update():
                        plot_widget.data[4].x = list(range(len(cpu_data)))
                        plot_widget.data[4].y = active_loads
                        plot_widget.data[5].x = list(range(len(cpu_data)))
                        plot_widget.data[5].y = interframe_loads
                
                frame_idx += 1
                time.sleep(0.1)  # 控制刷新频率（10Hz）
            except Exception as e:
                plot_status.value = f"Plot Error: {str(e)}"
                time.sleep(0.5)
    
    # 4. 按钮回调函数
    # 开始绘图
    def on_start_plot(_):
        global is_plotting
        if not is_connected:
            plot_status.value = "Error: Radar not connected"
            return
        is_plotting = True
        plot_status.value = "Plot Status: Running"
        plot_status.style.color = "green"
        # 启动绘图线程
        Thread(target=update_plots, daemon=True).start()
    
    # 停止绘图
    def on_stop_plot(_):
        global is_plotting
        is_plotting = False
        plot_status.value = "Plot Status: Stopped"
        plot_status.style.color = "red"
    
    # 开始录制
    def on_start_record(_):
        global is_recording
        if not is_connected:
            record_status.value = "Error: Radar not connected"
            return
        is_recording = True
        record_status.value = "Recording: Running (radar_data.log)"
        record_status.style.color = "green"
    
    # 停止录制
    def on_stop_record(_):
        global is_recording
        is_recording = False
        record_status.value = "Recording: Stopped"
        record_status.style.color = "red"
    
    # 绑定按钮回调
    start_plot_btn.on_click(on_start_plot)
    stop_plot_btn.on_click(on_stop_plot)
    start_record_btn.on_click(on_start_record)
    stop_record_btn.on_click(on_stop_record)
    
    # 5. 组装可视化界面
    control_box = HBox([
        start_plot_btn, stop_plot_btn, start_record_btn, stop_record_btn,
        plot_status, record_status
    ])
    
    full_visual_ui = VBox([
        Label(value="=== IWR6843AOP Radar Data Visualization ===", style={"font_size": "16px", "font_weight": "bold"}),
        control_box,
        plot_widget
    ])
    
    return full_visual_ui

# 显示可视化界面
visual_ui = create_visualization_ui()
display(visual_ui)